<a href="https://colab.research.google.com/github/SARU230/Matplot_lib_for-gromacs/blob/main/neutralize_charges_for_gromacs_itp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def distribute_residual_evenly(charges, residual, precision=10):
    """
    Distribute residual charge evenly and gradually over all atoms in small increments,
    to keep final sum exactly to target charge up to given precision.
    """
    n_atoms = len(charges)
    step = 10 ** (-precision)  # smallest increment per precision digit
    increments = int(round(residual / step))
    adjusted_charges = charges[:]
    idx = 0

    while increments != 0:
        increment_sign = 1 if increments > 0 else -1
        adjusted_charges[idx] += increment_sign * step
        increments -= increment_sign
        idx = (idx + 1) % n_atoms

    return adjusted_charges


def neutralize_charges_balanced(input_filename, output_filename, target_charge=0, precision=10):
    with open(input_filename, 'r') as f:
        charges = [float(line.strip()) for line in f if line.strip()]

    original_sum = sum(charges)
    n_atoms = len(charges)
    delta_q = target_charge - original_sum
    base_adjust = delta_q / n_atoms
    adjusted_charges = [q + base_adjust for q in charges]

    format_str = f"{{: >14.{precision}f}}"
    rounded_charges = [float(format_str.format(q)) for q in adjusted_charges]

    residual = target_charge - sum(rounded_charges)

    final_charges = distribute_residual_evenly(rounded_charges, residual, precision)
    final_sum = sum(final_charges)

    with open(output_filename, 'w') as f:
        for q in final_charges:
            f.write(format_str.format(q) + '\n')

    print(f"Original total charge: {original_sum:.12f}")
    print(f"Target charge: {target_charge}")
    print(f"Initial adjustment per atom: {base_adjust:.12e}")
    print(f"Rounding residual distributed: {residual:.16e}")
    print(f"Final total charge after redistribution: {final_sum:.{precision}f}")
    print(f"Balanced neutralized charges written to {output_filename}")


if __name__ == "__main__":
    input_file = input("Enter input charge filename: ").strip()
    output_file = input("Enter output filename for adjusted charges: ").strip()

    while True:
        try:
            target_charge = int(input("Enter the integer total molecular charge (e.g., 0, +1, -1): "))
            break
        except ValueError:
            print("Invalid input. Please enter an integer.")

    neutralize_charges_balanced(input_file, output_file, target_charge=target_charge, precision=10)


Enter input charge filename: cm5_charges.txt
Enter output filename for adjusted charges: neutralize.txt
Enter the integer total molecular charge (e.g., 0, +1, -1): 0
Original total charge: -0.000157000000
Target charge: 0
Initial adjustment per atom: 1.427272727273e-06
Rounding residual distributed: -2.9999999134194821e-09
Final total charge after redistribution: 0.0000000000
Balanced neutralized charges written to neutralize.txt


In [ ]:
# Replace 'output_charges.txt' with your actual filename
with open('neutralize.txt', 'r') as f:
    charges = [float(line.strip()) for line in f if line.strip()]

net_charge = sum(charges)
print(f"Net charge of output: {net_charge:.16f}")


Net charge of output: -0.0000000000000000
